In [1]:
#python のバージョンの表示
import sys
print(sys.version)

3.13.5 | packaged by Anaconda, Inc. | (main, Jun 12 2025, 16:37:03) [MSC v.1929 64 bit (AMD64)]


In [2]:
from ai import ai_match, ai2s

ai_match(ai=[ai2s, ai2s], match_num=5000)

ai2s VS ai2s


100%|██████████| 5000/5000 [00:07<00:00, 689.49it/s]

count     win    lose    draw
o        2908    1463     629
x        1439    2915     646
total    4347    4378    1275

ratio     win    lose    draw
o       58.2%   29.3%   12.6%
x       28.8%   58.3%   12.9%
total   43.5%   43.8%   12.8%



[('count',
  [{'win': 2908, 'lose': 1463, 'draw': 629},
   {'win': 1439, 'lose': 2915, 'draw': 646},
   {'win': 4347, 'lose': 4378, 'draw': 1275}],
  '7d'),
 ('ratio',
  [{'win': 0.5816, 'lose': 0.2926, 'draw': 0.1258},
   {'win': 0.2878, 'lose': 0.583, 'draw': 0.1292},
   {'win': 0.4347, 'lose': 0.4378, 'draw': 0.1275}],
  '7.1%')]

In [3]:
from marubatsu import Marubatsu

def judge(self):
    if self.move_count < self.BOARD_SIZE * 2 - 1:
        return Marubatsu.PLAYING
    # 〇 の勝利の判定
    if self.is_winner(Marubatsu.CIRCLE):
        return Marubatsu.CIRCLE
    # × の勝利の判定
    elif self.is_winner(Marubatsu.CROSS):
        return Marubatsu.CROSS   
    # 引き分けの判定
    elif self.is_full():
        return Marubatsu.DRAW
    # 上記のどれでもなければ決着がついていない
    else:
        return Marubatsu.PLAYING      
    
Marubatsu.judge = judge

In [4]:
from mbtest import test_judge

test_judge()

Start
test winner = playing
oooooooooo
test winner = o
ooooooooo
test winner = x
oooooooo
test winner = draw
o
Finished


In [5]:
ai_match(ai=[ai2s, ai2s], match_num=5000)

ai2s VS ai2s


100%|██████████| 5000/5000 [00:03<00:00, 1593.37it/s]

count     win    lose    draw
o        2884    1455     661
x        1439    2920     641
total    4323    4375    1302

ratio     win    lose    draw
o       57.7%   29.1%   13.2%
x       28.8%   58.4%   12.8%
total   43.2%   43.8%   13.0%



[('count',
  [{'win': 2884, 'lose': 1455, 'draw': 661},
   {'win': 1439, 'lose': 2920, 'draw': 641},
   {'win': 4323, 'lose': 4375, 'draw': 1302}],
  '7d'),
 ('ratio',
  [{'win': 0.5768, 'lose': 0.291, 'draw': 0.1322},
   {'win': 0.2878, 'lose': 0.584, 'draw': 0.1282},
   {'win': 0.4323, 'lose': 0.4375, 'draw': 0.1302}],
  '7.1%')]

In [6]:
def judge(self):
    if self.move_count < self.BOARD_SIZE * 2 - 1:
        return Marubatsu.PLAYING
    # 直前に着手を行ったプレイヤーの勝利の判定
    if self.is_winner(self.last_turn):
        return self.last_turn
    # 引き分けの判定
    elif self.is_full():
        return Marubatsu.DRAW
    # 上記のどれでもなければ決着がついていない
    else:
        return Marubatsu.PLAYING      
    
Marubatsu.judge = judge

In [7]:
test_judge()

Start
test winner = playing
oooooooooo
test winner = o
ooooooooo
test winner = x
oooooooo
test winner = draw
o
Finished


In [8]:
ai_match(ai=[ai2s, ai2s], match_num=5000)

ai2s VS ai2s


100%|██████████| 5000/5000 [00:02<00:00, 2094.84it/s]

count     win    lose    draw
o        2963    1420     617
x        1489    2876     635
total    4452    4296    1252

ratio     win    lose    draw
o       59.3%   28.4%   12.3%
x       29.8%   57.5%   12.7%
total   44.5%   43.0%   12.5%



[('count',
  [{'win': 2963, 'lose': 1420, 'draw': 617},
   {'win': 1489, 'lose': 2876, 'draw': 635},
   {'win': 4452, 'lose': 4296, 'draw': 1252}],
  '7d'),
 ('ratio',
  [{'win': 0.5926, 'lose': 0.284, 'draw': 0.1234},
   {'win': 0.2978, 'lose': 0.5752, 'draw': 0.127},
   {'win': 0.4452, 'lose': 0.4296, 'draw': 0.1252}],
  '7.1%')]

In [9]:
def move(self, x, y):
    if self.place_mark(x, y, self.turn):
        self.last_turn = self.turn
        self.turn = Marubatsu.CROSS if self.turn == Marubatsu.CIRCLE else Marubatsu.CIRCLE  
        self.move_count += 1
        self.last_move = x, y
        self.status = self.judge()
        if len(self.records) <= self.move_count:            
            self.records.append(self.last_move)
        else:
            self.records[self.move_count] = self.last_move
            self.records = self.records[0:self.move_count + 1]
            
Marubatsu.move = move

In [10]:
def is_winner(self, player):
    x, y = self.last_move
    if self.is_same(player, coord=[0, y], dx=1, dy=0) or \
       self.is_same(player, coord=[x, 0], dx=0, dy=1):
            return True
    # 左上から右下方向の判定
    if x == y and self.is_same(player, coord=[0, 0], dx=1, dy=1):
        return True
    # 右上から左下方向の判定
    if x + y == self.BOARD_SIZE - 1 and \
        self.is_same(player, coord=[self.BOARD_SIZE - 1, 0], dx=-1, dy=1):
        return True

    # どの一直線上にも配置されていない場合は、player は勝利していないので False を返す
    return False

Marubatsu.is_winner = is_winner

In [11]:
test_judge()

Start
test winner = playing
oooooooooo
test winner = o
ooooooooo
test winner = x
oooooooo
test winner = draw
o
Finished


In [12]:
ai_match(ai=[ai2s, ai2s], match_num=5000)

ai2s VS ai2s


100%|██████████| 5000/5000 [00:01<00:00, 2762.23it/s]

count     win    lose    draw
o        2958    1421     621
x        1469    2904     627
total    4427    4325    1248

ratio     win    lose    draw
o       59.2%   28.4%   12.4%
x       29.4%   58.1%   12.5%
total   44.3%   43.2%   12.5%



[('count',
  [{'win': 2958, 'lose': 1421, 'draw': 621},
   {'win': 1469, 'lose': 2904, 'draw': 627},
   {'win': 4427, 'lose': 4325, 'draw': 1248}],
  '7d'),
 ('ratio',
  [{'win': 0.5916, 'lose': 0.2842, 'draw': 0.1242},
   {'win': 0.2938, 'lose': 0.5808, 'draw': 0.1254},
   {'win': 0.4427, 'lose': 0.4325, 'draw': 0.1248}],
  '7.1%')]

In [13]:
def restart(self):
    self.initialize_board()
    self.turn = Marubatsu.CIRCLE     
    self.move_count = 0
    self.status = Marubatsu.PLAYING
    self.last_move = -1, -1          
    self.last_turn = None
    self.records = [self.last_move]
    self.rowcount = {
        Marubatsu.CIRCLE: [0] * self.BOARD_SIZE,
        Marubatsu.CROSS: [0] * self.BOARD_SIZE,
    }
    self.colcount = {
        Marubatsu.CIRCLE: [0] * self.BOARD_SIZE,
        Marubatsu.CROSS: [0] * self.BOARD_SIZE,
    }
    self.diacount = {
        Marubatsu.CIRCLE: [0] * 2,
        Marubatsu.CROSS: [0] * 2,
    }
    
Marubatsu.restart = restart

In [14]:
def move(self, x, y):
    if self.place_mark(x, y, self.turn):
        self.last_turn = self.turn
        self.turn = Marubatsu.CROSS if self.turn == Marubatsu.CIRCLE else Marubatsu.CIRCLE  
        self.move_count += 1
        self.last_move = x, y
        self.colcount[self.last_turn][x] += 1
        self.rowcount[self.last_turn][y] += 1
        if x == y:
            self.diacount[self.last_turn][0] += 1        
        if x + y == self.BOARD_SIZE - 1:
            self.diacount[self.last_turn][1] += 1        
        self.status = self.judge()
        if len(self.records) <= self.move_count:            
            self.records.append(self.last_move)
        else:
            self.records[self.move_count] = self.last_move
            self.records = self.records[0:self.move_count + 1]
            
Marubatsu.move = move

In [15]:
def unmove(self):
    if self.move_count > 0:
        x, y = self.last_move
        self.colcount[self.last_turn][x] -= 1
        self.rowcount[self.last_turn][y] -= 1
        if x == y:
            self.diacount[self.last_turn][0] -= 1        
        if x + y == self.BOARD_SIZE - 1:
            self.diacount[self.last_turn][1] -= 1           
        if self.move_count == 0:
            self.last_move = (-1, -1)
        self.move_count -= 1
        self.turn, self.last_turn = self.last_turn, self.turn
        self.status = Marubatsu.PLAYING
        x, y = self.records.pop()
        self.remove_mark(x, y)
        
        self.last_move = self.records[-1]  
        
Marubatsu.unmove = unmove

In [16]:
def is_winner(self, player):
    x, y = self.last_move
    if self.rowcount[player][y] == self.BOARD_SIZE or \
       self.colcount[player][x] == self.BOARD_SIZE:
        return True
    # 左上から右下方向の判定
    if x == y and self.diacount[player][0] == self.BOARD_SIZE:
        return True
    # 右上から左下方向の判定
    if x + y == self.BOARD_SIZE - 1 and \
        self.diacount[player][1] == self.BOARD_SIZE:
        return True

    # どの一直線上にも配置されていない場合は、player は勝利していないので False を返す
    return False

Marubatsu.is_winner = is_winner

In [17]:
test_judge()

Start
test winner = playing
oooooooooo
test winner = o
ooooooooo
test winner = x
oooooooo
test winner = draw
o
Finished


In [18]:
ai_match(ai=[ai2s, ai2s], match_num=5000)

ai2s VS ai2s


100%|██████████| 5000/5000 [00:01<00:00, 2523.63it/s]

count     win    lose    draw
o        2912    1445     643
x        1398    2970     632
total    4310    4415    1275

ratio     win    lose    draw
o       58.2%   28.9%   12.9%
x       28.0%   59.4%   12.6%
total   43.1%   44.1%   12.8%



[('count',
  [{'win': 2912, 'lose': 1445, 'draw': 643},
   {'win': 1398, 'lose': 2970, 'draw': 632},
   {'win': 4310, 'lose': 4415, 'draw': 1275}],
  '7d'),
 ('ratio',
  [{'win': 0.5824, 'lose': 0.289, 'draw': 0.1286},
   {'win': 0.2796, 'lose': 0.594, 'draw': 0.1264},
   {'win': 0.431, 'lose': 0.4415, 'draw': 0.1275}],
  '7.1%')]

In [19]:
def judge(self):
    if self.move_count == 0:
        return Marubatsu.PLAYING
    # 直前に着手を行ったプレイヤーの勝利の判定
    if self.is_winner(self.last_turn):
        return self.last_turn
    # 引き分けの判定
    elif self.is_full():
        return Marubatsu.DRAW
    # 上記のどれでもなければ決着がついていない
    else:
        return Marubatsu.PLAYING      
    
Marubatsu.judge = judge

In [20]:
test_judge()

Start
test winner = playing
oooooooooo
test winner = o
ooooooooo
test winner = x
oooooooo
test winner = draw
o
Finished


In [21]:
ai_match(ai=[ai2s, ai2s], match_num=5000)

ai2s VS ai2s


100%|██████████| 5000/5000 [00:02<00:00, 2318.97it/s]

count     win    lose    draw
o        2936    1416     648
x        1444    2937     619
total    4380    4353    1267

ratio     win    lose    draw
o       58.7%   28.3%   13.0%
x       28.9%   58.7%   12.4%
total   43.8%   43.5%   12.7%



[('count',
  [{'win': 2936, 'lose': 1416, 'draw': 648},
   {'win': 1444, 'lose': 2937, 'draw': 619},
   {'win': 4380, 'lose': 4353, 'draw': 1267}],
  '7d'),
 ('ratio',
  [{'win': 0.5872, 'lose': 0.2832, 'draw': 0.1296},
   {'win': 0.2888, 'lose': 0.5874, 'draw': 0.1238},
   {'win': 0.438, 'lose': 0.4353, 'draw': 0.1267}],
  '7.1%')]

In [22]:
def restart(self):
    self.initialize_board()
    self.turn = Marubatsu.CIRCLE     
    self.move_count = 0
    self.status = Marubatsu.PLAYING
    self.last_move = -1, -1          
    self.last_turn = None
    self.records = [self.last_move]
    
Marubatsu.restart = restart

In [23]:
def is_winner(self, player):
    x, y = self.last_move
    if self.is_same(player, coord=[0, y], dx=1, dy=0) or \
       self.is_same(player, coord=[x, 0], dx=0, dy=1):
            return True
    # 左上から右下方向の判定
    if x == y and self.is_same(player, coord=[0, 0], dx=1, dy=1):
        return True
    # 右上から左下方向の判定
    if x + y == self.BOARD_SIZE - 1 and \
        self.is_same(player, coord=[self.BOARD_SIZE - 1, 0], dx=-1, dy=1):
        return True

    # どの一直線上にも配置されていない場合は、player は勝利していないので False を返す
    return False

Marubatsu.is_winner = is_winner

In [24]:
def move(self, x, y):
    if self.place_mark(x, y, self.turn):
        self.last_turn = self.turn
        self.turn = Marubatsu.CROSS if self.turn == Marubatsu.CIRCLE else Marubatsu.CIRCLE  
        self.move_count += 1
        self.last_move = x, y
        self.status = self.judge()
        if len(self.records) <= self.move_count:            
            self.records.append(self.last_move)
        else:
            self.records[self.move_count] = self.last_move
            self.records = self.records[0:self.move_count + 1]
            
Marubatsu.move = move

In [25]:
def unmove(self):
    if self.move_count > 0:
        self.move_count -= 1
        self.turn, self.last_turn = self.last_turn, self.turn
        if self.move_count == 0:
            self.last_move = (-1, -1)
        self.status = Marubatsu.PLAYING
        x, y = self.records.pop()
        self.remove_mark(x, y)
        self.last_move = self.records[-1]  

Marubatsu.unmove = unmove

In [26]:
test_judge()

Start
test winner = playing
oooooooooo
test winner = o
ooooooooo
test winner = x
oooooooo
test winner = draw
o
Finished


In [27]:
ai_match(ai=[ai2s, ai2s], match_num=5000)

ai2s VS ai2s


100%|██████████| 5000/5000 [00:02<00:00, 2058.37it/s]

count     win    lose    draw
o        2878    1495     627
x        1451    2915     634
total    4329    4410    1261

ratio     win    lose    draw
o       57.6%   29.9%   12.5%
x       29.0%   58.3%   12.7%
total   43.3%   44.1%   12.6%



[('count',
  [{'win': 2878, 'lose': 1495, 'draw': 627},
   {'win': 1451, 'lose': 2915, 'draw': 634},
   {'win': 4329, 'lose': 4410, 'draw': 1261}],
  '7d'),
 ('ratio',
  [{'win': 0.5756, 'lose': 0.299, 'draw': 0.1254},
   {'win': 0.2902, 'lose': 0.583, 'draw': 0.1268},
   {'win': 0.4329, 'lose': 0.441, 'draw': 0.1261}],
  '7.1%')]